#### Introduction and Schema Diagram


In [8]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

#### Overview of the Data


In [9]:
%%sql
select name, type from sqlite_master where type in ('table', 'view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


#### Selecting New Albums to Purchase

In [10]:
%%sql

with usa_tracks_sold as 

(
  select il.* from invoice_line il 
    inner join invoice i on i.invoice_id = il.invoice_id
    inner join customer c on c.customer_id = i.customer_id
    where c.country = 'USA'
)

select g.name as genre, count(uts.invoice_line_id) as tracks_sold,
       cast(count(uts.invoice_line_id) as float)/(select count(*) from usa_tracks_sold) as percentage_sold
    
    from usa_tracks_sold uts inner join track t on t.track_id = uts.track_id
    inner join genre g on g.genre_id = t.genre_id 
    
    group by 1
    order by 2 DESC
    limit 10




Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

#### Analyzing Employee Sales Performance

In [11]:
%%sql

with customer_support_rep_sales as
(
select i.customer_id, c.support_rep_id, sum(i.total) as Total 
    from customer c inner join invoice i on i.customer_id = c.customer_id
    group by 1,2
)

select e.first_name || " " || e.last_name as employee,
       e.hire_date, sum(csrs.total) as total_sales 
       from customer_support_rep_sales csrs inner join employee e on 
        e.employee_id = csrs.support_rep_id
        group by 1;

Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

#### Analyzing Sales by Country

In [12]:
%%sql

with country_or_other as 

(
select 
    case
        when (
               select count (*) from customer where country = c.country ) = 1 then 'Other'
                

    else c.country
    end as country,
    c.customer_id,
    il.* 
    from invoice_line il 
    inner join invoice i on i.invoice_id = il.invoice_id 
    inner join customer c on c.customer_id = i.customer_id
    
)


select country, customers, total_sales, average_order, customer_lifetime_value 
from 
(
select country, count(distinct customer_id) as customers, sum(unit_price) as total_sales, 
    sum(unit_price)/count(distinct customer_id) as customer_lifetime_value, 
    sum(unit_price)/count(distinct invoice_id) as average_order,
    
    case 
    when country = 'Other' then 1
    else 0
    end as sort
    
    from country_or_other
    group by country
    order by sort asc, total_sales desc 

)
    
    




Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

#### Albums vs Individual Tracks

In [18]:
%%sql

with invoice_first_track as 

(

select il.invoice_id as invoice_id,
    min(il.track_id) as first_track_id
    from invoice_line il 
    group by 1
)


select album_purchase, count(invoice_id) as number_of_invoices, 
    cast(count(invoice_id) as float)/(select count (*) from invoice) as percent
    
from 
(
select ifs.*, case 
                 when 
                 (  
                 select t.track_id from track t
                     where t.album_id = ( select t2.album_id from track t2 
                                         where t2.track_id = ifs.first_track_id
                                        )
                 except
                     
                 select il2.track_id from invoice_line il2 
                     where il2.invoice_id = ifs.invoice_id
                 ) is null
    
             and 
             
             (
             select il2.track_id from invoice_line il2 
                     where il2.invoice_id = ifs.invoice_id             
             
    
            except
            
            
              select t.track_id from track t
                     where t.album_id = ( select t2.album_id from track t2 
                                         where t2.track_id = ifs.first_track_id
                                        )
            ) is null
            
           then 'Yes'
           else 'No'
           end as "album_purchase"
    from invoice_first_track ifs          
)
group by album_purchase



Done.


album_purchase,number_of_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.